In [28]:
import tweepy
import json
import pandas as pd
import csv

import mysql.connector
from mysql.connector import Error

In [29]:
#Twitter API credentials
consumer_key = 'pe7gsS8WNkANobhPvKU5q9PPv'
consumer_secret = 'EF8F0wezQz8HPBRp17DMXTEBrxTXJLBs1mzPnzjLLDlYsHql7D'
access_token = '468248438-aR7mCZ40MKTFq2TNIOGN0IWEegXUBGKDpN3QtDyT'
access_token_secret = 'K5dXjiphLnk17QfWTxigJHOJjgM3005T3c2OZj2X79FA8'

In [ ]:
#Definicion palabras claves busqueda
keywords = ['ODS', 'sostenibilidad', 'desarrollo', 'sostenible', 'cooperación']


def connect(user_id, user_name, user_loc, user_follow_count,user_friends_count, user_fav_count,user_status_count,tweet_id,text,created_at,source,reply_id, reply_user_id,retweet_id,reply_count,retweet_count,favorite_count,quote_count, hashtags, mentions,quote_id):
    """
    connect to MySQL database and insert twitter data
    """
    
    con = mysql.connector.connect(host = 'localhost',
                                  database='twitter', user='root', password = 'password', charset = 'utf8',auth_plugin='mysql_native_password')
    cursor = con.cursor()
    try:

        if con.is_connected():
            """
            Insert twitter data
            """
            
          
        
            query = "INSERT INTO users (user_id, user_name, user_loc, user_follow_count,user_friends_count, user_fav_count,user_status_count) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(query, (user_id, user_name, user_loc, user_follow_count,user_friends_count, user_fav_count,user_status_count))
            
            #DAVID!!! ojo, aqui pones users otra vez, deberia ser posts o como lo llames
            query2 = "INSERT INTO users (tweet_id,user_id,text,created_at,source,reply_id, reply_user_id,retweet_id,reply_count,retweet_count,favorite_count,quote_count, hashtags, mentions,quote_id ) VALUES (%s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(query2, (tweet_id,user_id,text,created_at,source,reply_id, reply_user_id,retweet_id,
                                    reply_count,retweet_count,favorite_count,quote_count,
                                    hashtags,mentions,quote_id))
            
            
            con.commit()
                     
            
            
    except Error as e:
        print(e)

    cursor.close()
    con.close()
    
    return

class MyStreamListener(tweepy.StreamListener):
    
    def on_data(self,data):
        # Twitter returns data in JSON format - we need to decode it first
        try:
            decoded = json.loads(data)
            
        except Exception as e:
            print ("Error on_data: %s" % str(e)) #we don't want the listener to stop
            return True
        
        #En caso de estar geolocalizado guardar la geolocalizacion
        #Si esta geolocalizado dentro de un bounding box (no exacta)
        if decoded.get('place') is not None:
            place_id = decoded.get('place').get('id')
            place_name =decoded.get('place').get('name')
        else:
            place_id = 'None'
            place_name = 'None'
            
        #Si es localizacion exacta
        #Geo is deprecated, they suggest to use simply coordinates
        if decoded.get('cordinates') is not None:
            m_coord = decoded.get('coordinates')
            c=0
            coord=''
            for i in range(0, len(m_coord)-1):
                mc=m_coord[i]
                m_coord=coord+mc+';'#use a different separator!
                c=c+1
            mc=m_coord[c]
            m_coord=coord+mc
        else:
            coord = 'None'
        
        #USER METADATA
        user_name = '@' + decoded.get('user').get('screen_name') #nombre cuenta @itdUPM 
        user_id=decoded.get('user').get('id') #id de la cuenta (int)
        user_loc=decoded.get('user').get('location') 
        user_follow_count=decoded.get('user').get('followers_count')
        user_friends_count=decoded.get('user').get('friends_count')
        user_fav_count=decoded.get('user').get('favourites_count')
        user_status_count=decoded.get('user').get('statuses_count')
        
        #POST METADATA
        created_at = decoded.get('created_at') #Fecha
        text = decoded['text'].replace('\n',' ') #Contenido tweet
        tweet_id = decoded['id'] #tweet id (int64)
        source = decoded['source'] #string source (web client, android, iphone) interesante???
        

        #REPLY METADATA
        reply_id=decoded['in_reply_to_status_id']
        reply_user_id=decoded['in_reply_to_user_id']
        #in_reply_to_screen_name Si es una respuesta screen_name al que responde

        
        #RETWEET
        
        if decoded.get('retweeted_status') is not None:
            retweet_id = decoded['retweeted_status'] ['id']
            
            #Carlota: Si es un retweet los campos de nº de retweets favs etc vienen dentro de retweeted status
            
            reply_count = decoded['retweeted_status']['reply_count'] #Number of times this Tweet has been replied to
            retweet_count = decoded['retweeted_status']['retweet_count'] #Number of times this Tweet has been retweeted
            favorite_count = decoded['retweeted_status']['favorite_count'] #how many times this Tweet has been liked by Twitter users.
            quote_count = decoded['retweeted_status']['quote_count']
            hashtags_list=decoded.get('retweeted_status').get('entities').get('hashtags')     
            mentions=decoded.get('retweeted_status').get('entities').get('user_mentions')
         
            if decoded['retweeted_status']['is_quote_status']:
                quote_id=decoded['retweeted_status']['quoted_status_id']
            else:
                quote_id='None'
                   
        
        else:
            reply_count = decoded['reply_count'] #Number of times this Tweet has been replied to
            retweet_count = decoded['retweet_count'] #Number of times this Tweet has been retweeted
            favorite_count = decoded['favorite_count'] #how many times this Tweet has been liked by Twitter users.
            quote_count = decoded['quote_count']
            retweet_id = 'None'
            
            
        #CAREFUL, THIS IS A LIST THAT MAY VARY THE LENGTH
            hashtags_list=decoded.get('entities').get('hashtags')     
            mentions=decoded.get('entities').get('user_mentions')
            
            print(decoded['is_quote_status'])
            if decoded['is_quote_status']:
                quote_id=decoded['quoted_status_id']
            else:
                quote_id='None'
            
            print(quote_id)
                
        
        hashtags=''
        c=0
        if len(hashtags_list)>0:
            for i in range(0, len(hashtags_list)-1):
                mh=hashtags_list[i].get('text')
                hashtags=hashtags+mh+';'
                c=c+1
            mh=hashtags_list[c].get('text')
            hashtags=hashtags+str(mh)
        else:
            hashtags='None'

        mention_ids=''
        c=0
        if len(mentions)>0:
            for i in range(0, len(mentions)-1):
                mid=mentions[i].get('id_str')
                mention_ids=mention_ids+mid+';'#use a different separator!
                c=c+1
            mid=mentions[c].get('id_str')
            mention_ids=mention_ids+str(mid)
        else:
            mentions_ids='None'
            
    
    

        #David: I think "," is better separator
        #Check the number of %s!!
        tweet = ['%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s\n' % (
                                    tweet_id,created_at,text,source,
            
                                    user_name,user_id, 
                                    user_loc, user_follow_count, user_friends_count, user_fav_count, 
                                    user_status_count, 
                                                                                                
                                    reply_count,retweet_count,favorite_count,quote_count, reply_id, reply_user_id,
                                    retweet_id, 
                                    quote_id,
                                    place_id, place_name, coord, hashtags, mentions )]
        
        """        # Writing tweets data
        with open('OutputStreamingKeywords_3.txt', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(tweet)
        
        return True"""
    
        #insert data just collected into MySQL database
        connect(user_id, user_name, user_loc, user_follow_count,user_friends_count, user_fav_count,user_status_count,tweet_id,text,created_at,source,reply_id, reply_user_id,retweet_id,reply_count,retweet_count,favorite_count,quote_count, hashtags, mentions,quote_id)
        print("Tweet colleted at: {} ".format(str(created_at)))
        
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

    # returning non-False reconnects the stream, with backoff.

    
"""# Writing csv titles
with open('OutputStreamingKeywords_3.txt', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['tweet_id','user_name','user_id','created_at','text','source', 'user_loc', 'user_follow',
                                      'user_friends', 'user_fav_count', 'user_status_count',
                                      'reply_count','retweet_count','favorite_count','quote_count', 'reply_id',
                                      'reply_user_id','retweet_id', 'quote_id','place_id',
                                      'place_name','coord','hashtags','mentions'])"""
    
if __name__ == '__main__':
    print ('Starting')
    #authorize twitter, initialize tweepy        
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    #create the api and the stream object
    myStreamListener = MyStreamListener()
    myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
    #Filter the stream by keywords
    myStream.filter(track = keywords)



Starting
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:34:26 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:34:26 +0000 2020 
SúperMujerRural

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:34:29 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:34:29 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:34:30 +0000 2020 
None
20646711
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:34:30 +0000 2020 
True
1227017049479581697
None
57437582;2191032740
Failed processing format-parameters; Python 'list' cannot 

False
None
ULTIMAHORA;AFP

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:04 +0000 2020 
False
None
None
82119937
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:04 +0000 2020 
False
None
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:06 +0000 2020 
False
None
Agenda2030
2886172323
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:06 +0000 2020 
None
20646711
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:06 +0000 2020 
False
None
None
205779753
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:06 +0000 2020 
False
None
None

Failed proces

None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:46 +0000 2020 
None
386100342
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:46 +0000 2020 
False
None
SanAntonio;Pelancura

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:48 +0000 2020 
Ahora
102094084
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:49 +0000 2020 
True
1227213374972420096
11Feb

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:50 +0000 2020 
None
818303870
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:35:53 +0000 2020 
False
None
None
1058470597
Failed processing format-par

ULTIMAHORA;AFP

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:36:35 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:36:35 +0000 2020 
None
335141638
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:36:36 +0000 2020 
False
None
None
4275477155
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:36:37 +0000 2020 
False
None
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:36:37 +0000 2020 
None
171047449
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:36:37 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted

False
None
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:37:16 +0000 2020 
True
1227198111841443840
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:37:16 +0000 2020 
ElGobiernoDeTodos

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:37:17 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:37:17 +0000 2020 
None
56410958
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:37:17 +0000 2020 
None
1025037163291914240;299317548
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:37:17 +0000 2020 
False
None
None

Failed processing format-paramete

...

False
None
None
824730500917063682
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:38:31 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:38:32 +0000 2020 
IP

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:38:34 +0000 2020 
None
96144763;289603372;29701712
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:38:34 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:38:36 +0000 2020 
False
None
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:38:37 +0000 2020 
False
None
None
15896476
Failed processing format-parameters; Python 'l

False
None
ItesalVentanas

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:39:13 +0000 2020 
False
None
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:39:13 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:39:13 +0000 2020 
None
802482424970149888
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:39:13 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:39:14 +0000 2020 
AlMomento

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:39:14 +0000 2020 
GobiernoDeCalle;10Feb;14Feb
856073126
Failed processing format-parameters; Python

False
None
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:39:57 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:39:58 +0000 2020 
Cuba
1025037163291914240;1075882433776676865
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:39:58 +0000 2020 
False
None
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:39:58 +0000 2020 
False
None
Ciencia

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:40:00 +0000 2020 
False
None
11Feb

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:40:00 +0000 2020 
False
None
None

Failed processing format-paramet

None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:40:43 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:40:43 +0000 2020 
None
226720055
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:40:44 +0000 2020 
False
None
None
521611526
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:40:44 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:40:45 +0000 2020 
False
None
None
1273687490;271922810
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:40:46 +0000 2020 
False
None
None
381725074
Failed processing format-parameters; Python 'list'

None
149991703;4515126989
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:41:36 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:41:36 +0000 2020 
None

Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:41:38 +0000 2020 
False
None
None
1009536394417631233;1252764865
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:41:39 +0000 2020 
False
None
None
2394855529
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:41:40 +0000 2020 
None
137908875
Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type
Tweet colleted at: Tue Feb 11 13:41:41 +0000 2020 
FelizMartes;Ciencia

Failed processing format